# download glove embedding

In [ ]:
!wget https://gist.githubusercontent.com/bastings/b094de2813da58056a05e8e7950d4ad1/raw/3fbd3976199c2b88de2ae62afc0ecc6f15e6f7ce/glove.840B.300d.sst.txt


--2021-01-16 17:37:20--  https://gist.githubusercontent.com/bastings/b094de2813da58056a05e8e7950d4ad1/raw/3fbd3976199c2b88de2ae62afc0ecc6f15e6f7ce/glove.840B.300d.sst.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53933315 (51M) [text/plain]
Saving to: ‘glove.840B.300d.sst.txt’

glove.840B.300d.sst 100%[===================>]  51.43M  87.2MB/s    in 0.6s    

2021-01-16 17:37:21 (87.2 MB/s) - ‘glove.840B.300d.sst.txt’ saved [53933315/53933315]



In [ ]:
!cp glove.840B.300d.sst.txt sample_data/glove.840B.300d.sst.txt

#import necessary packages






In [ ]:
import pandas as pd
import numpy as np
import os
import pickle
import numpy as np
import re
import itertools
from collections import Counter
#from New_Utils import *
# from keras.layers import Input, Dense, Embedding, Conv2D, MaxPool2D, LSTM
# from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.callbacks import ModelCheckpoint, EarlyStopping
# from keras.optimizers import Adam
from keras.optimizers import *
from keras.models import Model
from sklearn.model_selection import train_test_split
# from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from numpy import array
from keras.models import Sequential
# from keras.layers import Dense
from keras.layers import *
# from keras.layers.convolutional import Conv1D
# from keras.layers.convolutional import MaxPooling1D
# from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.regularizers import *
import tensorflow as tf
# from keras.layers.normalization import BatchNormalization
from keras.preprocessing.text import *
from keras.preprocessing.sequence import pad_sequences
#from Text_Preprocessing import *
import timeit

Using TensorFlow backend.


#load necessary supporting functions

In [ ]:
def dumpPickle(path,object_to_dump):
	f=open(path,'wb')
	pickle.dump(object_to_dump,f)
	f.close()

#Reads an object stores at 'path'
def readPickle(path):
	f=open(path,'rb')
	ob_to_load=pickle.load(f)
	f.close()
	return ob_to_load


#Loads an embedding matrix stores at emb_pickle_path if doesn't exists a pickled file, then creates One which is initialized by normal distribution with mean and std dev of the embeddings used, if given the option to save it saves
#the newly created embedding matrix at saveName
def load_create_embedding_matrix(word_index,vocab_size,emb_dim,emb_path,emb_pickle_path=False,save=False,saveName=None):
	if not emb_pickle_path:
		embedding_dict={}
		f=open(emb_path,'rb')
		for line in f:
			fields=line.split()
			word=fields[0]
			w_e=np.asarray(fields[1:],dtype='float32')
			embedding_dict[word]=w_e
		f.close()
		allembs=np.stack(embedding_dict.values())
		emb_mean,emb_std=allembs.mean(),allembs.std()
		embedding_matrix=np.random.normal(emb_mean,emb_std,(vocab_size,emb_dim))
		for word,index in word_index.items():
			#print(word)
			vector=embedding_dict.get(word)
			if vector is not None:
				embedding_matrix[index]=vector
		if save:
			dumpPickle(saveName,embedding_matrix)
	else:
		f=open(emb_pickle_path,'rb')
		embedding_matrix=pickle.load(f)
		f.close()
	return embedding_matrix

#creates a tokenizer from training data file in csv format, if there is one it loads and returns
def load_create_tokenizer(train_data,tok_path=None,savetokenizer=False):
	if tok_path == None:
		tokenizer=Tokenizer()
		tokenizer.fit_on_texts(train_data)
		len(tokenizer.word_index)
		if savetokenizer:
			dumpPickle('./New_Tokenizer.tkn',tokenizer)
	else:
		tokenizer=readPickle(tok_path)
	return tokenizer

def load_create_padded_data(X_train,savetokenizer,padPath=None,isPaddingDone=False,maxlen=None,tokenizer_path=None,save_new_padded_data=False,path_for_new_data=False):
	#print(tokenizer_path)
	if not isPaddingDone:
		tokenizer=load_create_tokenizer(X_train,tok_path=tokenizer_path,savetokenizer=savetokenizer)
		#word_index=tokenizer.word_index
		X_train=tokenizer.texts_to_sequences(X_train)
		X_train=pad_sequences(X_train,maxlen=maxlen)
		if save_new_padded_data:
			dumpPickle(path_for_new_data,X_train)
	else:
		X_train=readPickle(padPath)
	return X_train

#load dataset : utterances and its corresponding labels

In [ ]:
from google.colab import files 
  
  
uploaded = files.upload()

Saving TWEET_DATA_IEEE_TCSS - Sheet1.csv to TWEET_DATA_IEEE_TCSS - Sheet1.csv


In [ ]:
import io 
df = pd.read_csv(io.BytesIO(uploaded['TWEET_DATA_IEEE_TCSS - Sheet1.csv']), header= None)
df.head()

,0,1
0,RT @Bipartisanism: Katie Couric just schooled ...,STM
1,RT @stevesilberman: If you missed it: Sick bur...,EXP
2,@jk_rowling & the Never-Ending Story: With a #...,QUE
3,RT @somebadideas: The emotional & personal eff...,EXP
4,RT @jk_rowling: A 9-year-old Nigerian girl has...,STM


In [ ]:
text=df[0].tolist()
labels=df[1].tolist()

In [ ]:
print(len(df))
print(len(text))
print(len(labels))

7500
7500
7500


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_Text, test_Text, train_l, test_label = train_test_split(text, labels, test_size=0.20, random_state=42)

In [ ]:
train_Text = [s.strip() for s in train_Text]
test_Text = [s.strip() for s in test_Text]

text = train_Text + test_Text
max1=0

for i in range(0,len(text)):
	data=text[i].split(" ")
	max2=len(data)
	if(max2>max1):
		max1=max2

sequence_length = max1

#create padded data and embedding matrix

In [ ]:
print("creating data")
tokenizer=load_create_tokenizer(train_Text,None,True)
X_train=load_create_padded_data(X_train=train_Text,savetokenizer=False,isPaddingDone=False,maxlen=sequence_length,tokenizer_path='./New_Tokenizer.tkn')
X_test=load_create_padded_data(X_train=test_Text,savetokenizer=False,isPaddingDone=False,maxlen=sequence_length,tokenizer_path='./New_Tokenizer.tkn')
word_index=tokenizer.word_index
embedding_matrix=load_create_embedding_matrix(word_index,len(word_index)+1,300,'./glove.840B.300d.sst.txt',False,True,'./Emb_Mat.mat')

print("data created")

creating data
data created


In [ ]:
#word_index
print(X_train.shape)
print(X_test.shape)
print(embedding_matrix.shape)
#X_train[25]

(6000, 57)
(1500, 57)
(15154, 300)


# create one-hot encoded labels 

In [ ]:

lbl_dict={}
index=0
# u_label = list(set(train_label))
for dial_lbls in train_l:
	if dial_lbls not in lbl_dict:
		lbl_dict[dial_lbls]=index
		index=index+1
# for i,j in enumerate(u_label):
#   lbl_dict[j]=i

print(len(lbl_dict))



def create_label(label):
	
    Y=[]
    for i in label:
    	xxx=np.zeros(len(lbl_dict))
    	j=lbl_dict.get(i)
    	xxx[j]=1
    	Y.append(xxx)
    return Y

label = train_l
Y_train = create_label(label)

label = test_label
Y_test = create_label(label)

y_train=np.array([i for i in Y_train])
y_test=np.array([i for i in Y_test])

embedding_dim = 300

7


In [ ]:
print(y_test.shape)
print(y_test[24])

(1500, 7)
[1. 0. 0. 0. 0. 0. 0.]


In [ ]:
print(lbl_dict)
print(list(set(train_l)))
print(list(set(test_label)))
print(list(set(test_label)-set(train_l)))

{'SUG': 3, 'OTH': 5, 'REQ': 6, 'STM': 1, 'THT': 4, 'QUE': 2, 'EXP': 0}
['SUG', 'OTH', 'REQ', 'STM', 'THT', 'QUE', 'EXP']
['OTH', 'SUG', 'REQ', 'STM', 'THT', 'QUE', 'EXP']
[]


#create model

In [ ]:
print("Creating Model...")
inputs = Input(shape=(sequence_length,), dtype='int32')
embedding = Embedding(input_dim=len(word_index)+1, output_dim=embedding_dim, weights=[embedding_matrix], input_length=sequence_length)(inputs)
conv_0 = Conv1D(filters=64, kernel_size=10, padding='same', kernel_initializer='normal', activation='relu')(embedding)
maxpool_0 = MaxPooling1D(pool_size=2)(conv_0)
dropout = Dropout(0.1)(maxpool_0)
conv_1 = Conv1D(filters=128, kernel_size=10, padding='same', kernel_initializer='normal', activation='relu')(dropout)
maxpool_1 = MaxPooling1D(pool_size=2)(conv_1)
flatten = Flatten()(maxpool_1)
predictions = Dense(1000, activation='relu')(flatten)
predictions1 = Dense(len(lbl_dict), activation='softmax')(predictions)
adam = Adam(lr=0.01, decay=0.3)
model = Model(inputs=inputs, outputs=predictions1)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=4, batch_size=64, validation_data=(X_test, y_test))


predicted=model.predict(X_test)
#print("DONE")

Creating Model...


/usr/local/lib/python2.7/dist-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 6000 samples, validate on 1500 samples
Epoch 1/4
6000/6000 [==============================] - 26s 4ms/step - loss: 2.8189 - accuracy: 0.5223 - val_loss: 1.2119 - val_accuracy: 0.5800
Epoch 2/4
6000/6000 [==============================] - 25s 4ms/step - loss: 1.1173 - accuracy: 0.6073 - val_loss: 1.1558 - val_accuracy: 0.6000
Epoch 3/4
6000/6000 [==============================] - 26s 4ms/step - loss: 1.0237 - accuracy: 0.6377 - val_loss: 1.1301 - val_accuracy: 0.6080
Epoch 4/4
6000/6000 [==============================] - 25s 4ms/step - loss: 0.9559 - accuracy: 0.6555 - val_loss: 1.1137 - val_accuracy: 0.6127


In [ ]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 57)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 57, 300)           4546200   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 57, 64)            192064    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 28, 64)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 28, 64)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 28, 128)           82048     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 14, 128)           0   

#evaluate model

In [ ]:
Y_predicted=[]
for i in predicted:
    pos=i.argmax()
    Y_predicted.append(pos)
    
    
    
Y_test=[]
for i in y_test:
    pos=i.argmax()
    Y_test.append(pos)
    
    
    
from sklearn.metrics import accuracy_score
# from sklearn.naive_bayes import GaussianNB
# from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
# from sklearn.model_selection import cross_val_predict
# from sklearn.model_selection import train_test_split

print(accuracy_score(Y_test, Y_predicted))
print(classification_report(Y_test, Y_predicted,digits=4,))
print(confusion_matrix(Y_test, Y_predicted))

0.6126666666666667
              precision    recall  f1-score   support

           0     0.6340    0.9305    0.7541       806
           1     0.5254    0.4655    0.4936       333
           2     0.0000    0.0000    0.0000       132
           3     0.0000    0.0000    0.0000        75
           4     0.0000    0.0000    0.0000        53
           5     0.6364    0.2258    0.3333        62
           6     0.0000    0.0000    0.0000        39

   micro avg     0.6127    0.6127    0.6127      1500
   macro avg     0.2565    0.2317    0.2259      1500
weighted avg     0.4836    0.6127    0.5286      1500

[[750  54   0   0   0   2   0]
 [176 155   0   0   0   2   0]
 [119  12   0   0   0   1   0]
 [ 57  17   0   0   0   1   0]
 [  7  46   0   0   0   0   0]
 [ 38  10   0   0   0  14   0]
 [ 36   1   0   0   0   2   0]]


/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
